In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.registration import phase_cross_correlation


### Before running the code below, put all satellite patches together and replace the directory below 

In [ ]:
sat_image_root = './Satellite/' # path to the satellite patches
_, _, sat_filenames = next(os.walk(sat_image_root))

In [ ]:
sat_map = np.zeros([20000, 17000, 3])
image1 = cv2.imread(sat_image_root + '619400_5736195.png')
sat_map[600:1200, 600:1200, :] = image1
y = 900
x = 900
for j in range(280):
    print(j)
    if j %2 == 0:
        for i in range(0, 320):
            if j == 0 and i == 0:
                pass
            else:
                image2 = cv2.imread(sat_image_root + str(619400+int(j*5))+'_'+str(int(5736195-i*5))+'.png')
                if image2 is not None:
                    shift, _, _ = phase_cross_correlation(image1[:,:,0], image2[:,:,0])
                    y_offset, x_offset = shift
                    y += int(y_offset)
                    x += int(x_offset)
                    sat_map[y-300:y+300, x-300:x+300, :] = image2
                    image1 = image2
    else:
        for i in range(319,-1,-1):
            image2 = cv2.imread(sat_image_root + str(619400+int(j*5))+'_'+str(int(5736195-i*5))+'.png')
            if image2 is not None:
                shift, _, _ = phase_cross_correlation(image1[:,:,0], image2[:,:,0])
                y_offset, x_offset = shift
                y += int(y_offset)
                x += int(x_offset)
                sat_map[y-300:y+300, x-300:x+300, :] = image2
                image1 = image2




In [ ]:
# write the stitched satellite map to disk
cv2.imwrite('./satellite_map_new.png', sat_map)

In [ ]:
image_bl = cv2.imread(sat_image_root+'619400_5734600.png') # bottom left
image_br = cv2.imread(sat_image_root+'620795_5734600.png') # bottom right
image_tr = cv2.imread(sat_image_root+'620795_5736195.png') # top right
image_c = cv2.imread(sat_image_root+'620100_5735400.png') # center

In [ ]:
map_bl = sat_map[18168-300:18168+300, 492-300:492+300, :]
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(image_bl, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(map_bl.astype(np.uint8), cv2.COLOR_BGR2RGB))

shift, _, _ = phase_cross_correlation(map_bl[:,:,0], image_bl[:,:,0])
y_offset, x_offset = shift
print(y_offset, x_offset)

In [ ]:
map_br = sat_map[18528-300:18528+300, 15553-300:15553+300, :]
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(image_br, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(map_br.astype(np.uint8), cv2.COLOR_BGR2RGB))

shift, _, _ = phase_cross_correlation(map_br[:,:,0], image_br[:,:,0])
y_offset, x_offset = shift
print(y_offset, x_offset)

In [ ]:
map_tr = sat_map[1260-300:1260+300, 15966-300:15966+300, :]
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(image_tr, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(map_tr.astype(np.uint8), cv2.COLOR_BGR2RGB))

shift, _, _ = phase_cross_correlation(map_br[:,:,0], image_br[:,:,0])
y_offset, x_offset = shift
print(y_offset, x_offset)

In [ ]:
map_c = sat_map[9688-300:9688+300, 8255-300:8255+300, :]
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(cv2.cvtColor(image_c, cv2.COLOR_BGR2RGB))
plt.subplot(1,2,2)
plt.imshow(cv2.cvtColor(map_c.astype(np.uint8), cv2.COLOR_BGR2RGB))

shift, _, _ = phase_cross_correlation(map_c[:,:,0], image_c[:,:,0])
y_offset, x_offset = shift
print(y_offset, x_offset)

In [ ]:
# calculate the transformation between pixel coordinates and UTM coordinates

primary = np.array([[900., 900.], #tl
                    [492., 18168.], #bl
                    [15966., 1260.], #tr
                    [15553., 18528.], #br
                    [8255., 9688.]]) # c

secondary = np.array([[619400., 5736195.],
                      [619400., 5734600.],
                      [620795., 5736195.],
                      [620795., 5734600.],
                      [620100., 5735400.]])

# Pad the data with ones, so that our transformation can do translations too
n = primary.shape[0]
pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
unpad = lambda x: x[:,:-1]
X = pad(primary)
Y = pad(secondary)

# Solve the least squares problem X * A = Y
# to find our transformation matrix A
A, res, rank, s = np.linalg.lstsq(X, Y)

transform = lambda x: unpad(np.dot(pad(x), A))

print("Target:")
print(secondary)
print("Result:")
print(transform(primary))
print("Max error:", np.abs(secondary - transform(primary)).max())
